In [ ]:
import librosa
import pandas as pd
import numpy as np

In [ ]:
import glob

In [ ]:
pd.options.display.max_colwidth = 9999
train = pd.DataFrame({"path":glob.glob("/kaggle/input/dacon-audio/train/*/*")})

In [ ]:
train["ans"] = train["path"].apply(lambda x:x.split("/")[-2])
train

In [ ]:
# 데이터를 랜덤하게 추출 sample 함수
train = train.sample(8000)
train

In [ ]:
# 모든 데이터에 적용하기
from tqdm import tqdm
# mel_list = []
# for i in tqdm(train["path"]):
#     audio,sr = librosa.load(i,duration = 5, res_type = "kaiser_fast")
#     mel = librosa.feature.melspectrogram(audio)
#     mel_list.append(mel)
# mfcc와 같은 방식이되, 대신 mfcc는 압축을 좀 더 많이 하는 느낌이고, melspectogram이 이미지 크기가 좀 더 큼

In [ ]:
mel_list = np.load("../input/realmel/mel_list.npy") # mel_list 설명 + 점수 좋지 않음

In [ ]:
# mel_list = np.array(mel_list, dtype = np.float32).reshape(-1,128,216,1)
# mel_list.shape

In [ ]:
# np.save("mel_list",mel_list)

In [ ]:
train["ans"].value_counts()

In [ ]:
train["ans"] = train["ans"].replace({"africa":0,"australia":1,"canada":2,"england":3,"hongkong":4,"us":5})

In [ ]:
test = pd.DataFrame({"path":glob.glob("/kaggle/input/dacon-audio/test/*")})
test

In [ ]:
test["num"] = test["path"].apply(lambda x:x.split('/')[-1].split('.')[0]).astype("int")
test = test.sort_values("num").reset_index(drop=True)
test

In [ ]:
from tqdm import tqdm
# test_mel = []
# for i in tqdm(test["path"]):
#     audio, sr = librosa.load(i,duration = 5) 
#     mel = librosa.feature.melspectrogram(audio)
#     test_mel.append(mel)

In [ ]:
#test_mel = np.array(test_mel, dtype = np.float32).reshape(-1,128,216,1)

In [ ]:
#np.save("test_mel",test_mel)

In [ ]:
test_mel = np.load("../input/realmel/test_mel.npy")

In [ ]:
from keras import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import Adam

In [ ]:
best_score = 0
result = 0
i = 0
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 82)
for train_index, valid_index in skf.split(mel_list, train["ans"]):
    i += 1
    
    print(f"{i}회")
    X_train, X_valid = mel_list[train_index], mel_list[valid_index]
    y_train, y_valid = train["ans"].iloc[train_index], train["ans"].iloc[valid_index]
    es = EarlyStopping(patience = 20, verbose = 1)
    mc = ModelCheckpoint(f"best{i}.h5",save_best_only = True, verbose = 1) 
    rl = ReduceLROnPlateau(patience = 15, verbose = 1,factor = 0.25,min_lr = 1e-6) 
    model = Sequential()
    model.add(Conv2D(16,(2,2),activation = "elu",input_shape = (20,216,1), padding = "same"))
    model.add(MaxPool2D())
    model.add(Conv2D(32,(2,2),activation = "elu",padding = "same"))
    model.add(MaxPool2D())
    model.add(Conv2D(64,(2,2),activation = "elu",padding = "same"))
    model.add(MaxPool2D())
    model.add(GlobalAvgPool2D()) # 상대적으로 지역적인 정보가 덜 날아감 flatten 대신
    model.add(Dense(6,activation = "softmax"))
    model.compile(metrics = ["acc"], optimizer = Adam(lr = 0.0001), loss = "sparse_categorical_crossentropy") 
    history = model.fit(X_train,y_train,epochs = 500,validation_data = (X_valid,y_valid) ,callbacks = [es,mc,rl])
    
    model.load_weights(f"best{i}.h5")
    result += model.predict(test_mel, verbose = 1) / 5
    best_score += np.min(history.history["val_loss"]) / 5

In [ ]:
best_score

In [ ]:
result

In [ ]:
sub = pd.read_csv("/kaggle/input/dacon-audio/sample_submission.csv")
#sub.iloc[:,1:] = result
# sub1 = pd.read_csv("")
# sub2 = pd.read_csv("")
# sub.iloc[:,1:] = sub1.iloc[:,1:] * 0.7 + sub1.iloc[:,1:] * 0.3
sub.to_csv("sub_mel.csv",index=False)